# 올리브영 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd
import numpy as np
import re
import os
import time

In [2]:
global REV, OTHER

REV = pd.DataFrame(columns = ['reviewer_id', 'reviewer', 'info', 'review_ct', 'total_help', 'point', 'rank', 'total_review']) 
OTHER = pd.DataFrame(columns = ['reviewer', 'item_id', 'item', 'item_link', 'option', 'comment', "poll", 'star', 'help_ct', 'date', 'tag'])

In [3]:
chrome_path = '../../chromedriver'

## 제품 리뷰어 정보

In [4]:
# 2 3 4 ... 10 11
#  3 4 .... 10 11 12
#  3 4 ... 10

In [5]:
# p = 2
# while True:
#     try:
#         print(p)
#         driver.find_element_by_css_selector(f'#gdasContentsArea > div > div.pageing > a').click()
#         p += 1
#     except Exception as e:
#         print(e)
#         print(p)
#         break

In [6]:
# 리뷰어 정보
def reviewer_info(r):
    # r번째 리뷰어 정보 1~10
    driver.find_element_by_css_selector(f"#gdasList > li:nth-child({r}) > div.info > div > p.info_user > a.id").click()
    global Profile
    time.sleep(0.3)
    Profile = driver.find_element_by_css_selector('#Contents > div > div.user_lbox.user_box > div.info_user').text.split("\n")
    global Cnt
    Cnt = int(driver.find_element_by_css_selector('#Contents > div > div.user_rbox > p > span').text.replace(",", ""))

    REV.loc[len(REV)] = [
        int(re.findall("=(.*)", driver.current_url)[0]), # id
        Profile[0], # 리뷰어
        " | ".join(Profile[1:]), # 기타 정보
        int(re.sub("[^0-9]+", "", driver.find_element_by_css_selector('li.icon_01 > span').text)), # 리뷰수
        int(re.sub("[^0-9]+", "", driver.find_element_by_css_selector('li.icon_02 > span').text)), # 도움수
        int(re.sub("[^0-9]+", "", driver.find_element_by_css_selector('li.icon_03 > span').text)), # 점수
        int(re.sub("[^0-9]+", "", driver.find_element_by_css_selector('li.icon_04 > span').text)), # 랭킹
        Cnt # 누적 리뷰수
    ]

## 리뷰어가 쓴 다른 리뷰들

In [7]:
def other_reviews(p):
    # p번째 pop-up
    try:
        driver.find_element_by_css_selector(f'#reviewerGdasList > li:nth-child({p}) > a > div.view_info > div.txt_inner').click()
        time.sleep(0.3)
        item = driver.find_element_by_css_selector('#layerWrap850 > div > div > div.lay_recom_area > div > a').get_attribute('onclick').split(";")
    except:
        # pop-up 없음
        return 


    # 옵션 없는 경우
    try:
        option = driver.find_element_by_css_selector('div.lay_review_cont.scrbar p.item_option').text
    except:
        option = np.NaN

    # 상품 추천 내용 없는 경우 : 피부타입 건성에 좋아요~~~
    try:
        recm = driver.find_element_by_css_selector('div.lay_review_cont.scrbar div.poll_sample').text
    except:
        recm = np.NaN

    # tag 없는 경우
    try:
        tag = driver.find_element_by_css_selector('#layerWrap850 > div > div > div.layViewDetail.clrfix > div.inner02 > div.lay_review_cont.scrbar > p.tag').text.replace("\n", " | ")
    except:
        tag = np.NaN

    OTHER.loc[len(OTHER)] = [
        Profile[0], # 리뷰어
        re.findall("[(]'(.*?)',", item[1])[0], # 상품번호
        driver.find_element_by_css_selector('#layerWrap850 > div > div > div.lay_recom_area > div > a > dl > dt').text, # 상품명
        re.findall("href='(.*)'", item[0])[0], # 상품링크
        option, # 옵션, default: NaN
        driver.find_element_by_css_selector('div.inner02 div.txt_inner').text, # 리뷰 내용 # 재구매, 한달사용리뷰 앞에 붙음
        recm, # 상품 특징 정보
        int(driver.find_element_by_class_name('point').text[-2]), # 별점, 5점 만점에 x점
        int(driver.find_element_by_css_selector('div.recom_area span.num').text), # 도움 수
        driver.find_element_by_class_name('date').text, # 작성일자
        tag # 태그

    ]
    # 팝업 닫기
    driver.find_element_by_css_selector('#layerWrap850 > div > button').click()
    time.sleep(0.2)


In [8]:
if __name__ == "__main__":
    url_list = pd.read_csv("./url_list_0823.csv")['링크'].values
    driver = webdriver.Chrome(executable_path="../../chromedriver")

    for url in url_list[:1]:
        driver.get(url)
        # 리뷰 탭 클릭
        driver.find_element_by_css_selector('#reviewInfo > a').click()
        # 총 리뷰 수 (확인용)
        total = int(re.sub("[^0-9]+", "", driver.find_element_by_css_selector('#reviewInfo > a > span').text))
        # time.sleep(0.3)
        # paging
        # p = 2
        # while True:
            # try:
            # 일단 첫 페이지만
        for u in range(1, 11): # 한 페이지에 10명
            time.sleep(0.5)
            reviewer_info(u)
            # 끝까지 내리기
            ActionChains(driver).move_to_element(driver.find_element_by_id('Footer')).perform()
            # for r in range(1, Cnt+1): # 누적 리뷰수 만큼
            for r in range(5): # 일단 10개만
                other_reviews(r)
            # 뒤로가기 해서 새로운 리뷰어 선ㅌㅐㄱ
            driver.back()
            time.sleep(1)
        # driver.find_element_by_css_selector(f'#gdasContentsArea > div > div.pageing > a:nth-child({p})').click()
        # time.sleep(0.2)
                # p += 1

            # except: break
    driver.close()

In [9]:
REV

,reviewer_id,reviewer,info,review_ct,total_help,point,rank,total_review
0,9926765,화장품분석녀,"TOP 400 | 메이크업, 스킨케어 분야 탑리뷰어 | 복합성 | 웜톤 | 각질 | 모공",88,119,101049,385,243
1,9910724,앙꼬빵,지성 | 모공 | 블랙헤드,13,11,8029,2790,10
2,9506241,다블리,"TOP 50 | 건강/위생용품, 스킨케어 분야 탑리뷰어 | 복합성 | 웜톤 | 모공...",215,138,259893,33,522
3,7811054,라당,TOP 1000 | 지성 | 웜톤 | 각질 | 모공,54,23,52387,922,120
4,9953897,탄산수맛있당,TOP 900 | 건성 | 쿨톤 | 각질 | 미백,47,38,56962,859,69
5,9302420,체리영,"TOP 200 | 향수/디퓨저, 바디케어 분야 탑리뷰어 | 건성 | 웜톤 | 각질 ...",118,110,136403,194,391
6,9272675,만월,TOP 600 | 복합성 | 웜톤 | 미백 | 민감성,132,27,79470,577,502
7,8311546,OGNA,트러블성 | 쿨톤 | 각질 | 모공,26,1,32116,1205,26
8,9390291,깐돌깐순,TOP 600 | 복합성 | 쿨톤 | 각질 | 잡티,71,56,80908,564,93
9,9298921,고인물메이트,TOP 700 | 스킨케어 분야 탑리뷰어 | 건성 | 봄웜톤 | 각질 | 모공,73,26,75873,610,86


In [10]:
OTHER

,reviewer,item_id,item,item_link,option,comment,poll,star,help_ct,date,tag
0,화장품분석녀,A000000136952,크리니크\n[1+1한정기획] 크리니크 쏙보습크림 (100아워),https://www.oliveyoung.co.kr/store/goods/getGo...,[옵션] 써지50ml 1+1,수십년동안 이크림은 계속나오고 있는데요\n세일이라 싸게사긴햇는데\n이제품이 요즘나오...,피부타입\n건성에 좋아요\n피부고민\n보습에 좋아요\n자극도\n보통이에요,1,154,2021.04.22,복합성 | 웜톤 | 각질 | 모공 | 미백 | 민감성 | 블랙헤드 | 잡티 | 주름...
1,화장품분석녀,A000000147975,바이오힐보\n바이오힐보 프로바이오덤 리프팅크림 더블기획50*2(탄탄크림),https://www.oliveyoung.co.kr/store/goods/getGo...,NaN,보타닉힐보는 몃년전새로 나왓을때 여러제품다 써봣고요\n품질성분들이 좋앗어요\n완벽하...,피부타입\n복합성에 좋아요\n피부고민\n주름/미백에 좋아요\n자극도\n자극이 느껴져요,1,92,2021.04.22,복합성 | 웜톤 | 각질 | 모공 | 미백 | 민감성 | 블랙헤드 | 잡티 | 주름...
2,화장품분석녀,A000000150945,넘버즈인\n넘버즈인 1번 잘가 각질 토너 200ml,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN,이정도성분은 다른저가 제품들도 기본적으로 닥토에들어가 잇어요\n용량도 이제품보다 용...,피부타입\n복합성에 좋아요\n피부고민\n진정에 좋아요\n자극도\n보통이에요,1,37,2021.04.22,복합성 | 웜톤 | 각질 | 모공 | 미백 | 민감성 | 블랙헤드 | 잡티 | 주름...
3,화장품분석녀,A000000150946,넘버즈인\n넘버즈인 2번 메이크업 찰떡 토너 200ml,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN,성분아주좋아요 보습에 아주좋은제품입니다\n문제는 저는 눈시림이 잇는데 이제품눈이 좀...,피부타입\n건성에 좋아요\n피부고민\n보습에 좋아요\n자극도\n보통이에요,1,36,2021.04.22,복합성 | 웜톤 | 각질 | 모공 | 미백 | 민감성 | 블랙헤드 | 잡티 | 주름...
4,앙꼬빵,A000000154787,라운드랩\n★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),https://www.oliveyoung.co.kr/store/goods/getGo...,NaN,전에 어찌어찌하다 자작나무 수분크림 샘플을 받아 잘 쓰고 있는 중에\n1+1행사한다...,피부타입\n지성에 좋아요\n피부고민\n보습에 좋아요\n자극도\n자극없이 순해요,5,8,2021.08.09,지성 | 모공 | 블랙헤드 | 주름 | 탄력 | 피지과다 | 홍조
5,앙꼬빵,A000000151981,코스알엑스\n★총 180매★코스알엑스 원스텝 오리지널 클리어 패드 90매 더블 기획,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN,첫 스킨패드로 쓴 제품인데 다른 패드도 써보고 돌고 돌아 다시 코스알엑스로 돌아왔어...,NaN,5,2,2021.08.09,지성 | 모공 | 블랙헤드 | 주름 | 탄력 | 피지과다 | 홍조
6,앙꼬빵,A000000150963,브링그린\n브링그린 사철쑥 카밍 약산성 토너 패드 75매 더블기획,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN,얼굴에 붉은기가 잘 올라오는 타입으로\n피부 진정 패드를 찾고 있다가 후기가 좋아 ...,NaN,5,1,2021.08.09,지성 | 모공 | 블랙헤드 | 주름 | 탄력 | 피지과다 | 홍조
7,앙꼬빵,A000000147975,바이오힐보\n바이오힐보 프로바이오덤 리프팅크림 더블기획50*2(탄탄크림),https://www.oliveyoung.co.kr/store/goods/getGo...,NaN,보타닉힐보 때부터 업그레이드 된 바이오힐보까지 사용해 온 찐 후기임..ㅋㅋ\n저는 ...,피부타입\n복합성에 좋아요\n피부고민\n보습에 좋아요\n자극도\n자극없이 순해요,5,1,2021.08.09,NaN
8,다블리,A000000145471,페이스인페이스\nFIF 병풀 센서티브 앰플마스크,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN,클린제품 찾는다면!!\n#FIF병풀센서티브 앰플마스크\n\n✔️제 피부는 복합성이고...,피부타입\n복합성에 좋아요\n피부고민\n진정에 좋아요\n자극도\n자극없이 순해요,5,51,2020.12.18,복합성 | 웜톤 | 모공 | 블랙헤드
9,다블리,A000000148064,아비브\n아비브 어성초 스팟 패드 카밍터치 더블 기획,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN,데일리 진정 케어 하고 싶다면\n\n#아비브 어성초 스팟. 패드 카밍 터치\n\n✔...,NaN,5,43,2020.12.18,복합성 | 웜톤 | 모공 | 블랙헤드
